In [ ]:
from preproc_tabular import get_tabular
from dataset_sales import getDataset
import sales_prediction as sp
from torch.optim.adamw import AdamW
from torch import nn
import torch
import os

img_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\ResizedImages\\'
tab_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\sales_anagrafica_final.csv'
aggregated_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_season.csv'
aggregated_month_path = 'C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\Dati\\aggregated_sales_month.csv'

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Sto usando {device}")
modello = sp.create_model(0).to(device)

batch_size = 128
epochs = 3
criterion = nn.MSELoss()
optimizer = AdamW(modello.parameters(), lr=1e-2) 
early_stop = sp.EarlyStopping(1, 0.01)

data, references, descriptions = get_tabular(tab_path)
train, val = getDataset(references, data, descriptions, img_path, batch_size)

mse_final, mae_final, r2_final = 0, 0, 0
modello_final = 0
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    sp.train_loop(train, modello, criterion, optimizer, batch_size, device)
    mse, mae, r2 = sp.validation_loop(val, modello, criterion, device)
    early_stp = early_stop(mse)
    if early_stp:
        print('Early Stop attivato.')
        break
    mse_final, mae_final, r2_final = mse, mae, r2
    modello_final = modello
    
print("Done!")

os.makedirs('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\noagg_2')
with open('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\noagg_2\\descrizione.txt', 'w') as file:
    file.write("Addestramento con dati non aggregati, adamw, batch 128, 3 epoche, lr 1e-2, early stopping con 1 epoca e delta 0.01, dropout 0.5." + '\n\n')
    file.write(f"Metriche alla fine:\nMSE: {mse_final}\nMAE: {mae_final}\nR2: {r2_final}")
model_sc = torch.jit.script(modello_final)
model_sc.save('C:\\Users\\GRVRLD00P\\Documents\\Progetto ORS\\results\\noagg_2\\weights.pt')